## DEV - Nuclear NICD Measurement

### Notes

- Outline of approach
    1. Nuclei segmentation based on His::GFP channel
        1. Gaussian smoothing
        2. Otsu thresholding
    2. Get cytoplasmic regions by dilation of nuclei
    3. Compute various measurements

### Prep

In [ ]:
# Imports

import os

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi

from skimage import io
from skimage.filters import threshold_otsu
from skimage.morphology import disk

from ipywidgets import interact

In [ ]:
# Input file path for test files

dpath = r'../Data/Images/HisGFP/5min export/1/'

fnames = {'nuc_on'   : r'1b-His-gfp timeseries_zstack evry 30 s for 10 min.tif',
          'nuc_off'  : r'1e-Final His-gfp.tif',
          'nicd_on'  : r'1a-mch-NICD-import_every min for 10 mins.tif',
          'nicd_off' : r'1d-Final mch-NICD.tif'}

slice_ann_fpath  = r'../Data/Images/HisGFP/5min export/matched_planes.txt'
slice_ann_sample = 1

In [ ]:
# Parameters

# Cropping of empty space
camera_noise  =  10  
min_line_pxls = 100

In [ ]:
# Load data

data = {}
for fkey in fnames:
    img = io.imread(os.path.join(dpath, fnames[fkey]))
    data[fkey] = img
    print(fkey, img.shape, img.dtype)

In [ ]:
# Select relevant data

# Matching slices from nuc stacks
slice_ann = np.loadtxt(slice_ann_fpath, skiprows=1, delimiter=',', dtype=int)
target_slice = slice_ann[slice_ann_sample-1,1] - 1
print('Using target slice:', target_slice)

# Only every other tp in nuc on
data['nuc_on']  = data['nuc_on'][::2, target_slice, :,:]

# Only the first tp in nuc off
data['nuc_off'] = data['nuc_off'][0, target_slice, :,:]

for fkey in data:
    print(fkey, data[fkey].shape)

In [ ]:
# Crop top and bottom
# Note: For consistency, use nuc_on to generate the mask for all images

def crop_camera(img, ref_img, camera_noise, min_line_pxls):
    
    print('  Before:', img.shape)
    
    if ref_img.ndim == 2:
        crop_mask = np.sum(ref_img > camera_noise, axis=1) >= min_line_pxls
    elif ref_img.ndim == 3:
        crop_mask = np.sum(ref_img[0] > camera_noise, axis=1) >= min_line_pxls
    else:
        raise Exception("Unanticipated dimensionality of ref_img:", ref_img.ndim)
        
    img = img[..., crop_mask, :]
    
    print('  After: ', img.shape)
    
    return img
    
ref_img = data['nuc_on'].copy()
for fkey in data:
    print(fkey)
    data[fkey] = crop_camera(data[fkey], ref_img, camera_noise, min_line_pxls)

In [ ]:
# Show time course

@interact(ch=['nuc_on','nuc_off','nicd_on','nicd_off'], 
          t=(0, data['nuc_on'].shape[0]-1, 1), 
          show=False)
def show_timecourse(ch='nuc_on', t=0, show=False):
    if show:
        
        fig = plt.figure(figsize=(12,12))
        
        if 'on' in ch:
            plt.imshow(data[ch][t], cmap='gray')
        else:
            plt.imshow(data[ch], cmap='gray')
            
        plt.title('raw')
        plt.show()        

### Gaussian Smoothing

In [ ]:
# Apply smoothing

data['nuc_on_smooth'] = np.array([ndi.gaussian_filter(data['nuc_on'][t], 2) 
                                  for t in range(data['nuc_on'].shape[0])])

data['nuc_off_smooth'] = ndi.gaussian_filter(data['nuc_off'], 2)

In [ ]:
# Show time course

@interact(t=(0, data['nuc_on'].shape[0]-1, 1), show=False)
def show_timecourse(t=0, show=False):
    if show:
        fig = plt.figure(figsize=(12,12))
        plt.imshow(data['nuc_on_smooth'][t], cmap='gray')
        #plt.imshow(data['nuc_off_smooth'], cmap='gray')
        plt.title('smooth')
        plt.show()        

### Otsu Thresholding

In [ ]:
# Analyze histograms

# Init
fig = plt.figure(figsize=(16,4))

# Useful color list
viridis_t = plt.cm.get_cmap('viridis', data['nuc_on'].shape[0]+1).colors

# Histogram of `on` over time
for t in range(data['nuc_on'].shape[0]):
    plt.hist(data['nuc_on_smooth'][t].flatten(), 
             bins=255, range=(0, 255), 
             color=viridis_t[t], alpha=0.2,
             histtype='step', lw=5)
    
# Histogram of `off`
plt.hist(data['nuc_off_smooth'].flatten(), 
         bins=255, range=(0, 255), 
         color=viridis_t[-1], alpha=0.7,
         histtype='step', lw=2)

# Otsu thresholds for `on` over time
for t in range(data['nuc_on'].shape[0]):
    plt.vlines(threshold_otsu(data['nuc_on_smooth'][t].flatten()), 15000, 25000, 
               color=viridis_t[t], alpha=0.2)

# Otsu threshold for `off`
plt.vlines(threshold_otsu(data['nuc_off_smooth'].flatten()), 15000, 25000, 
           color=viridis_t[-1], alpha=1.0)

# Global Otsu threshold for `on`
plt.vlines(threshold_otsu(data['nuc_on_smooth'].flatten()), 0, 20000, 
           color='r', alpha=0.8)

# Cosmetics
plt.xlabel('Intensity (smoothed)')
plt.ylabel('Number of pixels')
#plt.xlim(0, 50)
plt.show()

# ->> Looks like using a global otsu threshold is appropriate.

In [ ]:
# Apply Otsu thresholding

threshold = threshold_otsu(data['nuc_on_smooth'].flatten())
data['nuc_on_mask']  = data['nuc_on_smooth'] > threshold
data['nuc_off_mask'] = data['nuc_off_smooth'] > threshold

In [ ]:
# Show time course

@interact(t=(0, data['nuc_on'].shape[0]-1, 1), show=False)
def show_timecourse(t=0, show=False):
    if show:
        fig = plt.figure(figsize=(12,12))
        plt.imshow(data['nuc_on_mask'][t], cmap='gray')
        #plt.imshow(data['nuc_off_mask'], cmap='gray')
        plt.title('thresh')
        plt.show()

### Approximation of Cytoplasmic Region

Just dividing nuclear vs. the rest of the image would be dangerous here because of variable amounts of empty space!

In [ ]:
# Dilate nuclei and then remove them

data['all_on_mask'] = np.array([ndi.binary_dilation(data['nuc_on_mask'][t], 
                                                    structure=disk(10, dtype=np.bool))
                                for t in range(data['nuc_on'].shape[0])])
data['cyt_on_mask'] = data['all_on_mask'] ^ data['nuc_on_mask']


data['all_off_mask'] = ndi.binary_dilation(data['nuc_off_mask'], 
                                           structure=disk(10, dtype=np.bool))
data['cyt_off_mask'] = data['all_off_mask'] ^ data['nuc_off_mask']

In [ ]:
# Show time course

@interact(t=(0, data['nuc_on'].shape[0]-1, 1), show=False)
def show_timecourse(t=0, show=False):
    if show:
        
        fig = plt.figure(figsize=(12,12))
        
        plt.imshow(data['cyt_on_mask'][t], cmap='gray')
        plt.imshow(np.ma.array(data['nuc_on_mask'][t], 
                               mask=data['nuc_on_mask'][t]==0),
                   cmap='viridis', vmin=0)
        
        #plt.imshow(data['cyt_off_mask'], cmap='gray')
        #plt.imshow(np.ma.array(data['nuc_off_mask_cons'], 
        #                       mask=data['nuc_off_mask_cons']==0),
        #           cmap='viridis', vmin=0)
        
        plt.title('cyto')
        plt.show()

In [ ]:
# Additional improvement; 
# making the masks more conservative (by erosion/dilation) to avoid mistakenly 
# attributing signal from the wrong region at the nucleus-cytoplasm boundary

# Conservative nuclear mask
data['nuc_on_mask_cons'] = np.array([ndi.binary_erosion(data['nuc_on_mask'][t], 
                                                        structure=disk(5, dtype=np.bool))
                                     for t in range(data['nuc_on'].shape[0])])
data['nuc_off_mask_cons'] = ndi.binary_erosion(data['nuc_off_mask'], 
                                               structure=disk(5, dtype=np.bool))

# Conservative cytoplasmic mask
data['nuc_on_mask_dilated'] = np.array([ndi.binary_dilation(data['nuc_on_mask'][t], 
                                                            structure=disk(3, dtype=np.bool))
                                        for t in range(data['nuc_on'].shape[0])])
data['cyt_on_mask_cons'] = data['all_on_mask'] ^ data['nuc_on_mask_dilated']

data['nuc_off_mask_dilated'] = ndi.binary_dilation(data['nuc_off_mask'], 
                                                   structure=disk(3, dtype=np.bool))
data['cyt_off_mask_cons'] = data['all_off_mask'] ^ data['nuc_off_mask_dilated']

In [ ]:
# Show time course

@interact(t=(0, data['nuc_on'].shape[0]-1, 1), show=False)
def show_timecourse(t=0, show=False):
    if show:
        
        fig = plt.figure(figsize=(12,12))
        
        plt.imshow(data['cyt_on_mask_cons'][t], cmap='gray')
        plt.imshow(np.ma.array(data['nuc_on_mask_cons'][t], 
                               mask=data['nuc_on_mask_cons'][t]==0),
                   cmap='viridis', vmin=0)
        
        #plt.imshow(data['cyt_off_mask_cons'], cmap='gray')
        #plt.imshow(np.ma.array(data['nuc_off_mask_cons'], 
        #                       mask=data['nuc_off_mask_cons']==0),
        #           cmap='viridis', vmin=0)
        
        plt.title('cyto conservative')
        plt.show()

### Compute Measurements

In [ ]:
# Results dict
m = measurements = {}

# Get `on` measurements over time
measurements['nuc_on_nicd_sum']  = [data['nicd_on'][t][data['nuc_on_mask_cons'][t]].sum() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['nuc_on_nicd_mean'] = [data['nicd_on'][t][data['nuc_on_mask_cons'][t]].mean() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['nuc_on_area']      = data['nuc_on_mask'].sum(axis=(-1, -2))

measurements['cyt_on_nicd_sum']  = [data['nicd_on'][t][data['cyt_on_mask_cons'][t]].sum() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['cyt_on_nicd_mean'] = [data['nicd_on'][t][data['cyt_on_mask_cons'][t]].mean() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['cyt_on_area']      = data['cyt_on_mask'].sum(axis=(-1, -2))

measurements['all_on_nicd_sum']  = [data['nicd_on'][t][data['all_on_mask'][t]].sum() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['all_on_nicd_mean'] = [data['nicd_on'][t][data['all_on_mask'][t]].mean() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['all_on_area']      = data['all_on_mask'].sum(axis=(-1, -2))

measurements['img_on_nicd_sum']  = [data['nicd_on'][t].sum() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['img_on_nicd_mean'] = [data['nicd_on'][t].mean() 
                                    for t in range(data['nuc_on'].shape[0])]
measurements['img_on_area']      = data['all_on_mask'].size

# Get `off` measurements
measurements['nuc_off_nicd_sum']  = data['nicd_off'][data['nuc_off_mask_cons']].sum()
measurements['nuc_off_nicd_mean'] = data['nicd_off'][data['nuc_off_mask_cons']].mean()
measurements['nuc_off_area']      = data['nuc_off_mask'].sum()

measurements['cyt_off_nicd_sum']  = data['nicd_off'][data['cyt_off_mask_cons']].sum()
measurements['cyt_off_nicd_mean'] = data['nicd_off'][data['cyt_off_mask_cons']].mean()
measurements['cyt_off_area']      = data['cyt_off_mask'].sum()

measurements['all_off_nicd_sum']  = data['nicd_off'][data['all_off_mask']].sum()
measurements['all_off_nicd_mean'] = data['nicd_off'][data['all_off_mask']].mean()
measurements['all_off_area']      = data['all_off_mask'].sum()

measurements['img_off_nicd_sum']  = data['nicd_off'].sum()
measurements['img_off_nicd_mean'] = data['nicd_off'].mean()
measurements['img_off_area']      = data['all_off_mask'].size

# Convert lists to arrays
for mkey in measurements:
    if isinstance(m[mkey], list):
        m[mkey] = np.array(m[mkey])

### Check the Results

In [ ]:
m_types = ['nicd_sum', 'nicd_mean', 'area']

@interact(m_type=m_types)
def show_measurements(m_type='nicd_sum'):
    
    plt.figure(figsize=(8,6))
    
    plt.plot(m['nuc_on_'+m_type], label='nuc')
    plt.plot(m['cyt_on_'+m_type], label='cyt')
    plt.plot(m['all_on_'+m_type], label='all')
    #plt.plot(m['img_on_'+m_type], label='img')
    
    plt.scatter([15], m['nuc_off_'+m_type])
    plt.scatter([15], m['cyt_off_'+m_type])
    plt.scatter([15], m['all_off_'+m_type])
    #plt.scatter([15], m['img_off_'+m_type])
    
    plt.legend()
    plt.xlabel('time [min]')
    plt.ylabel(m_type+' [...]')
    plt.ylim([0, plt.gca().get_ylim()[1]])
    
    plt.show()
    
# ->> Ratios will need to be computed to correct for absolute intensities and cell movement.
#     This is done in the corresponding ANA notebook.